<font size="+3.8">Scanpy single-cell pre-processing</font>  
<font size="+1.5"></font>  

Aim: Preprocess mouse brain single-cell data from Vanlandewijck 2018 Nature 

Dataset: https://betsholtzlab.org/VascularSingleCells/database.html

In [ ]:
from datetime import date
date.today().strftime('%d/%m/%Y')

In [ ]:
import os
os.getlogin()

In [ ]:
import random
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
import seaborn as sns
import platform
from matplotlib.pyplot import rc_context

In [ ]:
import utils

In [ ]:
os.environ['CONDA_DEFAULT_ENV'] # conda env

In [ ]:
platform.platform()

In [ ]:
sc.settings.verbosity = 3

In [ ]:
main_dir='\\\srvisds1.srv.med.uni-muenchen.de\BD-Dichgans\SF' # Win
main_dir='/Volumes/BD-Dichgans/SF' # Mac

In [ ]:
dataset_name = "Vanlandewijck2018"
organism = "Mouse"

# Load data

## Annotated 
Downloaded from: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE98816 (normalized count matrix and Series Matrix File = metadata2) and https://figshare.com/collections/_/4077260 02/02/2023

### Vascular Cells

In [ ]:
ad = sc.read_text(os.path.join(main_dir, "P06_vasc_scRNAseq", "Vanlandewijck2018", "GSE98816_Brain_samples_normalized_counts_matrix.txt.gz")).transpose() # vascular cells

In [ ]:
ad

In [ ]:
# Add var

In [ ]:
ad.var.head()

In [ ]:
# not available

In [ ]:
# Add metadata

In [ ]:
meta = pd.read_csv(os.path.join(main_dir, "P06_vasc_scRNAseq", "Vanlandewijck2018", "Descriptionofeachsinglecellsamples.csv"), sep=";")

In [ ]:
meta.head()

In [ ]:
meta["tissue origin"].value_counts()

In [ ]:
meta = meta[meta["tissue origin"] == "Brain"]

In [ ]:
pd.crosstab(meta["Mouse ID"], meta["genotype"])

In [ ]:
meta["genotype"].value_counts()

In [ ]:
meta["annoated cell types"].value_counts()

In [ ]:
meta2 = pd.read_csv(os.path.join(main_dir, "P06_vasc_scRNAseq", "Vanlandewijck2018", "GSE98816_series_matrix.txt"), sep="\t", header=None).T

In [ ]:
new_header = meta2.iloc[0]
new_header = [s.lstrip('!') for s in new_header] # remove leading !
meta2 = meta2[1:]
meta2.columns = new_header
meta2.Sample_description = [s.lstrip('processed data column: ') for s in meta2.Sample_description] # remove leading text

In [ ]:
meta2=meta2.reset_index()
meta2=meta2.iloc[:,1:]

In [ ]:
meta2.head(3)

In [ ]:
meta2.columns

In [ ]:
meta2.Sample_description

In [ ]:
all(meta2.Sample_geo_accession == meta2.ID_REF)

In [ ]:
meta2["tissue origin"] = meta2['Sample_description'].str.split('.').str[0]

In [ ]:
meta2["tissue origin"].value_counts()

Note: Matches count of vascular cells from metadata 1 and anndata object

In [ ]:
len(meta2.Sample_geo_accession.unique())

All Sample_geo_accession are unique

In [ ]:
all(meta2.Sample_geo_accession.isin(meta["GSM ID"]))

All Sample_geo_accession are in GSM ID from metadata table 1. Now we can match meta2.Sample_description (anndata descriptors) to meta["annoated cell types"]

In [ ]:
# use only selected cols from meta2
meta2 = meta2[['Sample_title', 'Sample_geo_accession', 'tissue origin', 'Sample_description']]
meta2.head()

In [ ]:
# use only selected cols from meta2
meta = meta[['GSM ID','GSM name','organism','Mouse ID','strain','genotype','age','tissue origin','annoated cell types','BioSample ID','SRA ID','Sequenced Reads']]
meta.head()

In [ ]:
meta.shape # contains vascular cells + 250 astrocytes

In [ ]:
meta2.shape # contains vascular cells only

In [ ]:
meta_merged = meta.merge(meta2, how='inner', left_on="GSM ID", right_on="Sample_geo_accession")
meta_merged.head()

In [ ]:
# format ad.obs

In [ ]:
ad.obs.head()

This is the format of metadata in the anndata object. Corresponds to meta_merged.Sample_description

In [ ]:
ad.shape

In [ ]:
temp = ad.obs.reset_index(level=0)["index"].str.split(pat=".",expand=True)
temp

In [ ]:
temp["name"] = temp[0] + "." + temp[1]

In [ ]:
temp[0].value_counts()

In [ ]:
temp["tissue"] = temp[0]
temp["cell_id"] = temp[1]

In [ ]:
temp

In [ ]:
temp["cell_type_fine"] = temp.cell_id.str.split(pat="_",expand=True)[0]

In [ ]:
temp.cell_type_fine.value_counts()

In [ ]:
#temp[temp["cell_type_fine"] == "EC3"].cell_id.unique()

In [ ]:
pd.crosstab(temp["cell_type_fine"], temp["cell_id"])

In [ ]:
temp['cell_type_broad'] = temp['cell_type_fine'].str.replace('\d+', '')
temp.cell_type_broad.value_counts()

Note: Broad cell types match between metadata and anndata objects

In [ ]:
assert all(ad.obs.reset_index()["index"] == temp.name)
assert len(ad.obs) == len(temp)

In [ ]:
ad.obs = temp[["tissue","cell_id","cell_type_fine","cell_type_broad","name"]]

In [ ]:
# add metadata from meta_merged to anndata

In [ ]:
ad.obs

In [ ]:
ad.obs = ad.obs.merge(meta_merged, how='inner', left_on="name", right_on="Sample_description")

In [ ]:
ad.obs.index = ad.obs["cell_id"]
ad.obs.index.name = None

In [ ]:
ad.obs.head(3)

In [ ]:
ad.obs.columns

In [ ]:
pd.crosstab(ad.obs["cell_type_fine"], ad.obs["annoated cell types"])

In [ ]:
ad.obs["annoated cell types"].value_counts()

Check if data is normalized:

In [ ]:
ad.layers

In [ ]:
# show expression of 100 random genes (across all spots)
import random
import seaborn as sns
random_genes=random.sample(range(0, ad.X.shape[1]), 100)
adata_sub = ad[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007));
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
sns.histplot(ad.X.sum(1), bins=100, kde=False)

In [ ]:
exp.head()

Note: Data is normalized by definition (see file name)

Corresponds to https://betsholtzlab.org/VascularSingleCells/database.html, except for astrocytes

### Astrocytes

In [ ]:
ad_astro = sc.read_text(os.path.join(main_dir, "P06_vasc_scRNAseq", "Vanlandewijck2018", "GSE99058_Brain_AC_250samples_normalized_counts_matrix.txt.gz")).transpose() # astrocytes

In [ ]:
ad_astro

In [ ]:
# format ad_astro.obs

In [ ]:
ad_astro.obs.head()

This is the format of metadata in the anndata object. Corresponds to meta_merged.Sample_description

In [ ]:
ad_astro.shape

In [ ]:
temp = ad_astro.obs.reset_index(level=0)["index"].str.split(pat=".",expand=True)
temp

In [ ]:
temp["name"] = temp[0] + "." + temp[1]

In [ ]:
temp[0].value_counts()

In [ ]:
temp["tissue"] = temp[0]
temp["cell_id"] = temp[1]

In [ ]:
temp

In [ ]:
temp["cell_type_fine"] = temp.cell_id.str.split(pat="_",expand=True)[0]

In [ ]:
temp.cell_type_fine.value_counts()

wrong labels?

In [ ]:
#temp[temp["cell_type_fine"] == "EC3"].cell_id.unique()

In [ ]:
pd.crosstab(temp["cell_type_fine"], temp["cell_id"])

In [ ]:
temp['cell_type_broad'] = temp['cell_type_fine'].str.replace('\d+', '')
temp.cell_type_broad.value_counts()

Note: Broad cell types match between metadata and anndata objects

In [ ]:
assert all(ad_astro.obs.reset_index()["index"] == temp.name)
assert len(ad_astro.obs) == len(temp)

In [ ]:
ad_astro.obs = temp[["tissue","cell_id","cell_type_fine","cell_type_broad","name"]]

In [ ]:
ad_astro.obs["annoated cell types"] = "AC"

Check if data is normalized:

In [ ]:
ad_astro.layers

In [ ]:
# show expression of 100 random genes (across all spots)
import random
import seaborn as sns
random_genes=random.sample(range(0, ad.X.shape[1]), 100)
adata_sub = ad_astro[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007));
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
sns.histplot(ad_astro.X.sum(1), bins=100, kde=False)

In [ ]:
exp.head()

Note: Data is normalized by definition (see file name)

### Merge

Merge anndata objects from vasculature and astrocytes

But first, drop 18 ACs from vasculature dataset. In https://betsholtzlab.org/VascularSingleCells/database.html exactly 250 ACs are shown, presumably from the 250 AC dataset.

In [ ]:
ad = ad[~ad.obs['annoated cell types'].isin(['AC'])]

In [ ]:
ad

In [ ]:
ad_astro

In [ ]:
ad_merged = anndata.concat([ad, ad_astro], join="inner")

In [ ]:
ad_merged

In [ ]:
ad_merged.obs.head(3)

In [ ]:
ad_merged.obsm['X_umap'] # no umap available as expected

In [ ]:
ad_merged.obs["clusters_fine"] = ad_merged.obs["annoated cell types"]

In [ ]:
ad_merged.obs["clusters"] = ad_merged.obs["annoated cell types"]

In [ ]:
# merge subclusters
new_names = {
    'ECs': {'EC1','EC2','EC3','aEC','vEC','capilEC'},
    'Pericytes': {'PC'},
    'SMCs': {'aSMC','aaSMC','vSMC'},
    'Oligos': {'OL'},
    'Fibroblasts': {'FB1', 'FB2'},
    'Microglia': {'MG'},
    'Astrocytes': {'AC'}
}
# re-format
result = {}
for key, value in new_names.items():
    for elem in value:
        result[(elem)] = key
new_cluster_names = dict(sorted(result.items()))
print(new_cluster_names)

In [ ]:
ad_merged.obs['clusters'] = (
    ad_merged.obs['clusters']
    .map(new_cluster_names)
    .astype('category')
)

In [ ]:
# alternative: keep AV zonation

In [ ]:
# merge subclusters
new_names = {
    'ECs_non_AV': {'EC1','EC2','EC3'},
    'ECs_Arterial': {'aEC'},
    'ECs_Capillary': {'capilEC'},
    'ECs_Venous': {'vEC'},
    'Pericytes': {'PC'},
    'SMCs': {'aSMC','aaSMC','vSMC'},
    'Oligos': {'OL'},
    'Fibroblasts': {'FB1', 'FB2'},
    'Microglia': {'MG'},
    'Astrocytes': {'AC'}
}
# re-format
result = {}
for key, value in new_names.items():
    for elem in value:
        result[(elem)] = key
new_cluster_names = dict(sorted(result.items()))
print(new_cluster_names)

In [ ]:
# fine annotation
ad_merged.obs['clusters2'] = (
    ad_merged.obs['clusters_fine']
    .map(new_cluster_names)
    .astype('category')
)

In [ ]:
ad_merged.obs.clusters.value_counts()

In [ ]:
ad_merged.obs.clusters2.value_counts()

In [ ]:
# exclude clusters with <50 cells
cluster_counts = ad_merged.obs['clusters'].value_counts()
cluster_counts

In [ ]:
ad_merged = ad_merged[ad_merged.obs['clusters'].isin(cluster_counts[cluster_counts>49].index)]

Check if data is normalized:

In [ ]:
ad_merged.layers

In [ ]:
# show expression of 100 random genes (across all spots)
random_genes=random.sample(range(0, ad.X.shape[1]), 100)
adata_sub = ad_merged[:,random_genes]
exp=pd.DataFrame(np.matrix(adata_sub.X))
# plot
pl1=sns.displot(data=pd.melt(exp),x='value',height=4,hue='variable',kind="kde",warn_singular=False,legend=False,palette=list(np.repeat('#086da6',100)), lw=0.3) # genes with 0 expression are excluded
pl1.set(xlim=(-0.5, 7),ylim=(0,0.007));
plt.show()

In [ ]:
sns.set(rc={'figure.figsize':(4,4)})
pl=sns.histplot(data=pd.melt(exp),x='value',binwidth=0.5,legend=True,palette=list(np.repeat('#086da6',100)))
pl.set(xlim=(0, 10));
plt.show()

In [ ]:
sns.histplot(ad_merged.X.sum(1), bins=100, kde=False)

In [ ]:
exp.head()

# Log-normalize merged dataest

In [ ]:
ad_merged.layers["counts"] = ad_merged.X.copy() # save unnormalized raw RNA counts - retrieve via adata.X = adata.layers["counts"]

In [ ]:
sc.pp.normalize_total(ad_merged, inplace=True) # Normalize each spot by total counts over all genes, so that every spot has the same total count after normalization.

In [ ]:
sns.histplot(ad_merged.X.sum(1), kde=False)

In [ ]:
sc.pp.log1p(ad_merged) # X = log(X + 1)

In [ ]:
sns.histplot(ad_merged.X.sum(1), kde=False)

In [ ]:
ad_merged.layers["normalized"] = ad_merged.X.copy() # save normalized + log-transformed (but unscaled) counts - retrieve via adata.X = adata.layers["normalized"]

In [ ]:
# Identify highly-variable genes
sc.pp.highly_variable_genes(ad_merged)
sc.pl.highly_variable_genes(ad_merged)

In [ ]:
ad_merged.layers

In [ ]:
ad_merged.layers["normalized"].max()

# Focus on: Foxf2

In [ ]:
gene="Foxf2"

## Plot

In [ ]:
sns.set_theme(style='white')
sc.pl.matrixplot(ad_merged, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var")

In [ ]:
sc.pl.matrixplot(ad_merged, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var")

In [ ]:
sc.pl.dotplot(ad_merged, [gene], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var")

In [ ]:
sc.pl.dotplot(ad_merged, [gene], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var")

## Excl zonation

In [ ]:
utils.summarize_gene_expression(adata = ad_merged, gene = gene, groupby = "clusters", 
                          study_name = dataset_name, organism = organism,
                          export = True, output_dir = os.path.join(main_dir, "P06_Foxf2_per_celltype", "Foxf2_summarized")
                         )

## Incl zonation

In [ ]:
utils.summarize_gene_expression(adata = ad_merged, gene = gene, groupby = "clusters2", 
                          study_name = dataset_name, organism = organism,
                          export = True, output_dir = os.path.join(main_dir, "P06_Foxf2_per_celltype", "Foxf2_summarized", "incl_zonation")
                         )

# Focus on: Other genes

In [ ]:
target_genes = ["Foxo1", "Tek", "Nos3", "Htra1", "Egfl8", "Flt1", "Kdr", "Ptprb", "Nrp1", "Nrp2", "Efnb2", "Itgb1", "Itga6", "Angpt2", "Cdh5", "Cldn5", "Ocln", "Ctnnb1"]

## Excl zonation

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(ad_merged, gene, study_name = dataset_name, organism = organism, groupby = "clusters",
                                    output_dir=os.path.join(main_dir, "P06_Foxf2_per_celltype", "Other_genes_summarized"), export=True
                                   ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(ad_merged, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_merged, [target_genes[0]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(ad_merged, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_merged, [target_genes[1]], groupby='clusters', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

## Incl zonation

In [ ]:
other_genes_results = {
    gene: utils.summarize_gene_expression(ad_merged, gene, study_name = dataset_name, organism = organism, groupby = "clusters2",
                                    output_dir=os.path.join(main_dir, "P06_Foxf2_per_celltype", "Other_genes_summarized", "incl_zonation"), export=True
                                   ) for gene in target_genes
}

In [ ]:
# some plots

In [ ]:
sc.pl.matrixplot(ad_merged, [target_genes[0]], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_merged, [target_genes[0]], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.matrixplot(ad_merged, [target_genes[1]], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

In [ ]:
sc.pl.dotplot(ad_merged, [target_genes[1]], groupby='clusters2', swap_axes=False, figsize=(2,5), standard_scale="var", layer="normalized")

# Correlate gene expression (Foxf2 and Foxo1)

Using MAGIC denoising

In [ ]:
import magic
import scprep

In [ ]:
#sc.pp.scale(ad_merged)

In [ ]:
sns.set_theme(style='white')

## ECs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = ad_merged[ad_merged.obs.clusters == "ECs"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X, y=adata_EC[:,'Foxo1'].X, c=adata_EC[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((1,1), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## PCs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = ad_merged[ad_merged.obs.clusters == "Pericytes"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X, y=adata_EC[:,'Foxo1'].X, c=adata_EC[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((0,0), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## All cell types

In [ ]:
gg = ["Foxf2","Foxo1"]

In [ ]:
ad_merged

In [ ]:
#matrix = pd.DataFrame(adata.X) # not compatible with sparse 
matrix = ad_merged.X
#matrix.columns = ad_merged.var.index.tolist()

In [ ]:
cutoff_var = 700

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
ad_merged.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(ad_merged, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13, 6))

scprep.plot.scatter(x=ad_merged[:,'Foxf2'].X, y=ad_merged[:,'Foxo1'].X, c=ad_merged[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')

scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', title='After MAGIC')
plt.axline((1,1), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## capECs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = ad_merged[ad_merged.obs.clusters2 == "ECs_Capillary"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X, y=adata_EC[:,'Foxo1'].X, c=adata_EC[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((1,1), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## aECs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = ad_merged[ad_merged.obs.clusters2 == "ECs_Arterial"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X, y=adata_EC[:,'Foxo1'].X, c=adata_EC[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((1,1), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

## vECs

In [ ]:
gg = ["Foxf2","Foxo1","Nos3"]

In [ ]:
adata_EC = ad_merged[ad_merged.obs.clusters2 == "ECs_Venous"]

In [ ]:
adata_EC

In [ ]:
adata_EC.layers

In [ ]:
#matrix = pd.DataFrame(adata_EC.X) # not compatible with sparse 
matrix = adata_EC.X
matrix.columns = adata_EC.var.index.tolist()

In [ ]:
cutoff_var = None

In [ ]:
scprep.plot.plot_library_size(matrix, cutoff=cutoff_var)

In [ ]:
# filter lowly expressed genes and cells with a small library size
#matrix = scprep.filter.filter_library_size(matrix, cutoff=cutoff_var)
#matrix.head()

Note: Skipped normalization as data is already log-normalized

In [ ]:
adata_EC.layers

### Creating the MAGIC operator
If you don't specify parameters, MAGIC creates an operator with the following default values: knn=5, knn_max = 3 * knn, decay=1, t=3.

In [ ]:
magic_op = magic.MAGIC()

### Running MAGIC with gene selection
The magic_op.fit_transform function takes the normalized data and an array of selected genes as its arguments. If no genes are provided, MAGIC will return a matrix of all genes. The same can be achieved by substituting the array of gene names with genes='all_genes'.

In [ ]:
%%time
emt_magic = magic_op.fit_transform(adata_EC, genes=['Foxf2', 'Foxo1', 'Nos3'])

### Visualizing gene-gene relationships

We can see gene-gene relationships much more clearly after applying MAGIC. Note that the change in absolute values of gene expression is not meaningful - the relative difference is all that matters.

In [ ]:
np.corrcoef(emt_magic[:,['Foxf2','Foxo1']].X, rowvar = False)[0][1]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 6))
scprep.plot.scatter(x=adata_EC[:,'Foxf2'].X, y=adata_EC[:,'Foxo1'].X, c=adata_EC[:,'Nos3'].X, ax = ax1,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='Before MAGIC')
scprep.plot.scatter(x=emt_magic[:,'Foxf2'].X, y=emt_magic[:,'Foxo1'].X, c=emt_magic[:,'Nos3'].X, ax=ax2,
                    xlabel='Foxf2', ylabel='Foxo1', legend_title="Nos3", title='After MAGIC')
plt.axline((1,1), slope=1, color="black", alpha=0.3, linestyle="--")
plt.tight_layout()
plt.show()

# Session Info

In [ ]:
sc.logging.print_versions()